In [26]:
ROBOFLOW_API_KEY = "fuq6Zq7mEkfez2aEYNCQ"
ROBOFLOW_MODEL = "best.pt" # eg xx-xxxx--#
ROBOFLOW_SIZE = 416

import cv2
import base64
import numpy as np
import requests

In [27]:
upload_url = "".join([
    "https://detect.roboflow.com/",
    ROBOFLOW_MODEL,
    "?access_token=",
    ROBOFLOW_API_KEY,
    "&format=image",
    "&stroke=5"
])

In [28]:
video = cv2.VideoCapture(0)

In [41]:
# Infer via the Roboflow Infer API and return the result
def infer():
    # Get the current image from the webcam
    ret, img = video.read()
    # Resize (while maintaining the aspect ratio) to improve speed and save bandwidth
    height, width, channels = img.shape
    scale = ROBOFLOW_SIZE / max(height, width)
    img = cv2.resize(img, (round(scale * width), round(scale * height)))

    # Encode image to base64 string
    retval, buffer = cv2.imencode('.jpg', img)
    
    img_str = base64.b64encode(buffer)

    # Get prediction from Roboflow Infer API
    resp = requests.post(upload_url, data=img_str, headers={
        "Content-Type": "application/x-www-form-urlencoded"
    }, stream=True).raw

    print(resp.read())
    # Parse result image
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    print(image)


    return image

In [42]:
# Main loop; infers sequentially until you press "q"
while 1:
    # On "q" keypress, exit
    if(cv2.waitKey(0) == ord('q')):
        break

    # Synchronously get a prediction from the Roboflow Infer API
    image = infer()
    # And display the inference results
    cv2.imshow('image', image)
# Release resources when finished
video.release()
cv2.destroyAllWindows()


b'{"message":"Not Found"}'


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:816: error: (-215:Assertion failed) !buf.empty() in function 'cv::imdecode_'


In [46]:
import torch
import numpy as np
import cv2
from time import time


class MugDetection:
    """
    Class implements Yolo5 model to make inferences on a youtube video using Opencv2.
    """

    def __init__(self, capture_index, model_name):
        """
        Initializes the class with youtube url and output file.
        :param url: Has to be as youtube URL,on which prediction is made.
        :param out_file: A valid output file name.
        """
        self.capture_index = capture_index
        self.model = self.load_model(model_name)
        self.classes = self.model.names
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)

    def get_video_capture(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """
      
        return cv2.VideoCapture(self.capture_index)

    def load_model(self, model_name):
        """
        Loads Yolo5 model from pytorch hub.
        :return: Trained Pytorch model.
        """
        if model_name:
            model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_name, force_reload=True)
        else:
            model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
        return labels, cord

    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """
        return self.classes[int(x)]
    
    def automate_frame_split(self,frame, size):
        split_constant = size//3
        #print(split_constant)
        startpoint1 = (0,0)
        end_point1 = (size, split_constant)
        color1 = (255, 0, 0)
        thickness1 = 2

        start_point2 = (0, split_constant+1)
        end_point2 = (size, (2*split_constant+1))
        color2 = (226,181,0)
        thickness2 = 2

        start_point3 = (0, 2*(split_constant+1))
        end_point3 = (size, size)
        color3 = (0,0,255)
        thickness3 = 2

        frame = cv2.rectangle(frame, start_point1, end_point1, color1, thickness1)
        cv2.putText(frame, 'Too high', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        frame = cv2.rectangle(frame, start_point2, end_point2, color2, thickness2)
        cv2.putText(frame, 'Perfect', (20,split_constant+41), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        frame = cv2.rectangle(frame, start_point3, end_point3, color3, thickness3)
        cv2.putText(frame, 'Too Low', (20,2*split_constant+41), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        return frame
    
                   

    def g_plot_boxes(self, results, frame, size):
        split_constant = size//3
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """
        labels, cord = results
        center = ()
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.3:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                
                top_left = (x1, y1)
                bottom_right = (x2, y2) 
                
                x_mid = (top_left[0] + bottom_right[0])//2
                y_mid = (top_left[1] + bottom_right[1])//2
                center = (x_mid, y_mid)
                
                if(center[0] >= 0 and center[1] >= 0) and (center[0] <= size and center[1] <= split_constant):
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255,0,0), 2)
                    cv2.putText(frame, ''+self.class_to_label(labels[i])+': Too High ', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,0), 2)
                elif(center[0] >= 0 and center[1] >= (split_constant+1)) and (center[0] <= size and center[1] <= (2*split_constant+1)):
                    cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                    cv2.putText(frame, ''+self.class_to_label(labels[i])+': Perfect ', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)
                elif(center[0] >= 0 and center[1] >= (2*(split_constant+1))) and (center[0] <= size and center[1] <= size):
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 2)
                    cv2.putText(frame, ''+self.class_to_label(labels[i])+': Too Low ', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                    
                
                #cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)
                
                
                

        return frame
    

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """
        cap = self.get_video_capture()
        assert cap.isOpened()
        size = 900
      
        while True:
          
            ret, frame = cap.read()
            assert ret
            
            frame = cv2.resize(frame, (size, size))
            
            frame = self.automate_frame_split(frame, size)
            
            
            start_time = time()
            results = self.score_frame(frame)
            frame = self.g_plot_boxes(results, frame, size)
            
            
            end_time = time()
            fps = 1/np.round(end_time - start_time, 2)
            #print(f"Frames Per Second : {fps}")
             
#             cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
            
            cv2.imshow('YOLOv5 Detection', frame)
 
            if cv2.waitKey(1) & 0xFF == 27:
                break
      
        cap.release()
        cv2.destroyAllWindows()
    
        
        
# Create a new object and execute.
detector = MugDetection(capture_index=0, model_name='best.pt')
detector()

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\91845/.cache\torch\hub\master.zip
YOLOv5  2022-8-1 Python-3.8.5 torch-1.8.2+cu111 CUDA:0 (GeForce GTX 1650, 4096MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7249215 parameters, 0 gradients, 16.8 GFLOPs
Adding AutoShape... 


Using Device:  cuda


In [21]:
import cv2

cap = cv2.VideoCapture(0)
size = 416
while True:
    
    ret, frame = cap.read()
    frame = cv2.resize(frame, (416,416))
    frame = automate_frame_split(frame, 416)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
    
cap.release()
cv2.destroyAllWindows()

In [20]:
def automate_frame_split(frame, size):
    split_constant = size//3
    #print(split_constant)
    startpoint1 = (0,0)
    end_point1 = (size, split_constant)
    color1 = (255, 0, 0)
    thickness1 = 2
    
    start_point2 = (0, split_constant+1)
    end_point2 = (size, (2*split_constant+1))
    color2 = (226,181,0)
    thickness2 = 2
    
    start_point3 = (0, 2*(split_constant+1))
    end_point3 = (size, size)
    color3 = (0,0,255)
    thickness3 = 2
    
    frame = cv2.rectangle(frame, start_point1, end_point1, color1, thickness1)
    frame = cv2.rectangle(frame, start_point2, end_point2, color2, thickness2)
    frame = cv2.rectangle(frame, start_point3, end_point3, color3, thickness3)
    
    return frame
                    
    
    
    

138
